In [1]:
import pandas as pd
from IPython.display import display
import numpy as np
from collections import Counter
import nltk
import psycopg2
from nltk.corpus import stopwords
import re
from itertools import chain 
from IPython.display import Image
from sklearn.metrics.pairwise import cosine_similarity
import math

In [4]:
File = r'F:\Pretrained Models\glove.840B.300d\glove.840B.300d.txt'

def loadGloveModel(File):
    print("Loading Glove Model")
    f = open(File,'r',encoding="utf8")
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    print(len(gloveModel)," words loaded!")
    return gloveModel
def similarity(model,word1,word2):
    vec1 = [model[word1]]
    vec2 = [model[word2]]
    similarity_score = cosine_similarity(X =vec1, Y=vec2, dense_output=True)
    return similarity_score[0][0]
gloveModel = loadGloveModel(File)

Loading Glove Model


ValueError: could not convert string to float: '.'

In [78]:
def process(array):
    text = re.sub(r'\[[0-9]*\]',' ',str(array))  #Remove Numbers
    text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text) # Remove nums
    text = re.sub(r'\s+',' ',text)  #Remove extra space
    text = re.sub(r"[^a-zA-Z0-9]+",' ',text)  #Remove special characters
    text = text.lower()  #Lower case all
    text = nltk.sent_tokenize(text)  #Tokenize to sentences 
    keywords = [nltk.word_tokenize(sentence) for sentence in text]
    for i in range(len(keywords)):
        keywords[i] = [word for word in keywords[i] if word not in stopwords.words('english')]
    return keywords

#Frame pre-processing function
def prepro(frame,col_name):
    frame2 = frame
    for c in range(len(frame2)):
        keywords = process(frame2[col_name][c])
        frame2[col_name][c] = keywords
    return frame2

def flat(array):     # 2D array reduction
    flatten_list = [j for sub in array for j in sub] 
    flatten_list = list(chain.from_iterable(flatten_list))
    return flatten_list


# normalize() -> given an array, converts to 1/0, top int(pos) will be 1
def normalize(keys, pos =3):  
    ax = [i for i in keys]
    temp = [i for i in keys]
    temp.sort()
    temp = temp[-pos:]
    for x in temp:
        ax[keys.index(x)] = 1
    for x in range(len(ax)):
        if ax[x] != 1:
            ax[x] = 0
    return ax

# compute() => category[] to be called outside
def compute(caption,category,top =3):
    ar = []
    score = []
    
    # Code to get frequency distribution and unique keywords array
    counts = Counter(caption)
    labels, values = zip(*counts.items())
    ## sort your values in descending order
    indSort = np.argsort(values)[::-1]
    ## rearrange your data
    keywords = np.array(labels)[indSort]  # Label
    caption_freq = np.array(values)[indSort]  # Values
    
    # Detect words not in Google Dict | Put freq = 0
    for x in keywords:
        try:
            restConst = similarity(gloveModel,x,'something')
        except KeyError:
            caption_freq[np.where(keywords == x)] = 0
        
    #Google similaity function
    for x in category:
        empty = []
        for y in keywords:
            try:
                empty.append(similarity(gloveModel,x,y))
            except:
                empty.append(0)
        ar.append(empty)
    
    # Store the similarity values in dataframe
    frame = pd.DataFrame(ar, columns = keywords)
    
    #Normalize | top select
    for key in frame.columns:
        frame[key] = normalize(frame[key].tolist(),top)
    
    # Multiply with frequency
    for row in range(len(frame)):
        frame.values[row] = [i*j for i,j in zip(frame.values[row],caption_freq)]
    # Sum the values => Score
    for row in range(len(frame)):
        score.append(sum(frame.values[row]))
    
    frame['category'] = category
    frame['Scores'] = score
    return frame

def deviation(array):
    mu = max(array)
    l = len(array)
    ar = []
    for x in range(l):
        ar.append(math.sqrt((array[x]-mu)**2)/l)
    total = sum(ar)
    for x in range(l):
        ar[x] = (ar[x]/total)*100
    return ar

# shortlist category | deviation score, category list, thresold
def sort_cat(dev,cat, thre = 2):
    final_cat = []
    for i in range(len(dev)):
        if dev[i] <=thre:
            final_cat.append(cat[i])
    return final_cat

In [73]:
conn = psycopg2.connect(host="winkl-production.cpsgstz5cgos.us-west-2.rds.amazonaws.com",port="5432",database="winkl_production", user="winkl", password="Winkl911911")
cur = conn.cursor()

In [74]:
cat1 = ["food", "fashion", "beauty", "lifestyle", "traveler", "photography", "makeup", "art", "gaming", "fitness", "health", "entertainment", "technology", "finance", "luxury", "animal", "parenting", "books", "sports", "education"]
cat_should_be = ['food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal']
cat2 = ['adventure', 'blogging', 'casual', 'literature', 'international']
cat3 = ["food", "fashion", "beauty", "lifestyle", "travel", "photography", "makeup", 'girl', 'boy']
cat_food = ['fresh', 'spicy','masala','meal','food','foodie','delicious','recipe','foodlove','indianfood','foodlover','breakfast','eatclean','foodporn','vegetarian','foodpics','menu','foodgasm','sweet','healthy','dessert','chai','cheese','coffee','vegan','yummy']

272230, "Akshara Ashok", "https://www.instagram.com/happyfluffcomics"
287579, "Camera Setups 🎥", "https://www.instagram.com/camera_setups"   Camera blog
211647, "Ranveer Allahbadia ", "https://www.instagram.com/beerbiceps/"

270929, "RAJVEE GANDHI", "https://www.instagram.com/rxjvee"
215644, "Virat Kohli", "https://www.instagram.com/virat.kohli/"
112067, "MUMBAI ", "https://www.instagram.com/itz_mumbai/"
8633, "Archisha Tripathi", "https://www.instagram.com/foodparadise_india/"
165653, "Mythri Iyer ", "https://www.instagram.com/mythri_iyer/"

12359, "Devu And Tanu ", "https://www.instagram.com/thegluttonandtheowl/"   Fashion Beauty    
35043, "bhukkhadbybirth", "https://www.instagram.com/bhukkhadbybirth/"   Food blogger
301190, "https://www.instagram.com/glambypragya/"     makeup
301192, "https://www.instagram.com/financialtimes/"   finance
300226, "https://www.instagram.com/clever_canine/"    Pet animal
219002, "https://www.instagram.com/beauty.tipss01/"   Beauty tips
215620, "ARCHITECTURE HUNTER", "https://www.instagram.com/architecture_hunter/"   Architecture photos

105965


In [79]:
ids = [272230,287579,211647,270929,215644,112067,8633,165653,12359,35043,301190,301192,300226,219002,215620]
check_result = pd.DataFrame(columns = ['user_id','handle','url','caption amount','category from caption','category from bio'])
errlog = pd.DataFrame(columns = ['error'])
for x in ids:
    try:
        userid = x
        cat = cat_should_be
        cur.execute('SELECT "insta_user_posts"."insta_user_id", "insta_user_posts"."caption" FROM "insta_user_posts" WHERE (insta_user_id = '+str(userid)+')')
        randomframe = pd.DataFrame()
        randomframe = pd.DataFrame(cur.fetchall(),columns=["user_id", "Caption"])
        # display(randomframe.head(10))
        # print("Total captions in DB: ",len(randomframe),"\n")
        # if len(randomframe) == 0:
        #     break
        r2 = prepro(randomframe,'Caption')
        # Merge
        r3 = r2.groupby('user_id').agg(lambda x: x.tolist())
        # display(r3)

        # To 1D | Run once only
        r4 = r3
        r4['Caption'][userid] = flat(r3['Caption'][userid])
        # display(r4['Caption'])

        caption_array = r4['Caption'].tolist()
        # [[]] => []
        for x in caption_array:
            caption_array = x
        frame = pd.DataFrame()
        frame = pd.DataFrame(compute(caption_array,cat,3))

        # print(len(frame))
        dff = pd.DataFrame(frame)
        #Convert to Percentage
        per = dff['Scores'].tolist()
        per_sum = sum(per)
        for x in range(len(per)):
            per[x] = round((per[x]/per_sum)*100)
        dff['Percentage'] = per
        # dff

        cur.execute('SELECT  "instagram_accounts"."user_id","instagram_accounts"."handle","instagram_accounts"."url", "instagram_accounts"."bio" FROM "instagram_accounts" WHERE (user_id ='+str(userid)+')')
        biorandomframe = pd.DataFrame(cur.fetchall(),columns=["user_id","handle","url", "bio"])
        biorandomframe.head()
        bior2 = prepro(biorandomframe,'bio')
        bio_array = bior2['bio'].tolist()
        # [[]] => []
        bio_array = bio_array[0][0]
        # print(bio_array)
        bioframe = pd.DataFrame()
        bioframe = pd.DataFrame(compute(bio_array,cat,3))
        #Convert to Percentage
        biopercent = bioframe['Scores'].tolist()
        bioper_sum = sum(biopercent)
        for x in range(len(biopercent)):
            biopercent[x] = round((biopercent[x]/bioper_sum)*100)
        bioframe['Percentage'] = biopercent
        # display(bioframe)

        caption_deviation = deviation(dff['Scores'].tolist())
        bio_deviation = deviation(bioframe['Scores'].tolist())
        record = {'user_id':biorandomframe.at[0,'user_id'],'handle':biorandomframe.at[0,'handle'],'url':biorandomframe.at[0,'url'],'caption amount':len(randomframe),'category from caption':str(sort_cat(caption_deviation,cat,thre = 2.5)),'category from bio':str(sort_cat(bio_deviation,cat,thre = 2.5))}
        check_result = check_result.append(record,ignore_index=True)
    except Exception as Argument:
        errlog = errlog.append({'error':str(biorandomframe.at[0,'user_id'])+": "+str(Argument)},ignore_index=True)
        
display(check_result)
display(errlog)

C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is

,user_id,handle,url,caption amount,category from caption,category from bio
0,272230,happyfluffcomics,https://www.instagram.com/happyfluffcomics,55,"['food', 'beauty']","['art', 'technology']"
1,287579,camera_setups,https://www.instagram.com/camera_setups,51,"['food', 'sports', 'technology']",['entertainment']
2,211647,beerbicep,https://www.instagram.com/beerbiceps/,145,"['food', 'beauty', 'sports']","['beauty', 'sports', 'technology', 'education']"
3,215644,virat.kohli,https://www.instagram.com/virat.kohli/,206,"['food', 'beauty', 'sports']","['lifestyle', 'traveler', 'education']"
4,112067,itz_mumbai,https://www.instagram.com/itz_mumbai/,330,"['food', 'sports']","['fashion', 'art', 'technology']"
5,8633,foodparadise_india,https://www.instagram.com/foodparadise_india/,213,['food'],"['beauty', 'sports', 'technology']"
6,165653,mythri_iyer,https://www.instagram.com/mythri_iyer/,242,"['food', 'beauty']","['technology', 'finance']"
7,12359,thegluttonandtheowl,https://www.instagram.com/thegluttonandtheowl/,780,"['food', 'beauty']","['food', 'beauty']"
8,35043,bhukkhadbybirth,https://www.instagram.com/bhukkhadbybirth/,1008,['food'],"['food', 'fashion', 'beauty', 'art', 'entertai..."
9,301190,glambypragya,https://www.instagram.com/glambypragya/,12,"['food', 'beauty']","['fashion', 'beauty']"


,error
0,211647: index out of bounds


In [80]:
check_result.to_csv(r'E:\Winkl Mains\DATA\glove model testing\glove_twitter27B_200d.csv')

## END ##

### testing code below

In [39]:
### Caption categories

xx = dff['Scores'].tolist()
sd = deviation(xx)
row = {'category':cat,'actual':xx, 'deviation':sd}
devi = pd.DataFrame(row)
display(devi)

print(sort_cat(sd,cat,thre = 1.5))
print(sort_cat(sd,cat,thre = 2.5))
print(sort_cat(sd,cat,thre = 3.5))


,category,actual,deviation
0,food,738,1.394727
1,fashion,209,6.448223
2,makeup,197,6.562858
3,beauty,884,0.000000
4,lifestyle,89,7.594574
5,luxury,101,7.479939
6,traveler,102,7.470386
7,art,553,3.162018
8,photography,138,7.126481
9,fitness,56,7.909820


['food', 'beauty']
['food', 'beauty']
['food', 'beauty', 'art']


In [61]:
cur.execute('SELECT  "instagram_accounts"."user_id","instagram_accounts"."handle", "instagram_accounts"."bio" FROM "instagram_accounts" WHERE (user_id ='+str(userid)+')')
biorandomframe = pd.DataFrame(cur.fetchall(),columns=["user_id","handle", "bio"])
biorandomframe.head()
bior2 = prepro(biorandomframe,'bio')
bio_array = bior2['bio'].tolist()
# [[]] => []
bio_array = bio_array[0][0]
print(bio_array)
bioframe = pd.DataFrame()
bioframe = pd.DataFrame(compute(bio_array,cat,3))
#Convert to Percentage
biopercent = bioframe['Scores'].tolist()
bioper_sum = sum(biopercent)
for x in range(len(biopercent)):
    biopercent[x] = round((biopercent[x]/bioper_sum)*100)
bioframe['Percentage'] = biopercent
display(bioframe)

['collabs', 'queries', 'singermythri', 'gmail', 'com']


C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,com,gmail,singermythri,queries,collabs,category,Scores,Percentage
0,0,0,0,0,0,food,0,0
1,1,0,0,0,0,fashion,1,8
2,0,0,0,0,1,makeup,1,8
3,0,0,0,0,0,beauty,0,0
4,0,0,0,0,0,lifestyle,0,0
5,0,0,0,0,0,luxury,0,0
6,0,0,0,0,0,traveler,0,0
7,1,0,0,0,0,art,1,8
8,0,0,0,0,0,photography,0,0
9,0,0,0,0,0,fitness,0,0


In [41]:
### Bio categories
bio_cat_scores = bioframe['Scores'].tolist()
bio_dev_score = deviation(bio_cat_scores)
row_bio = {'category':cat,'actual':bio_cat_scores, 'deviation':bio_dev_score}
bio_devi_frame = pd.DataFrame(row_bio)
display(bio_devi_frame)

print(sort_cat(bio_dev_score,cat,thre = 1.5))
print(sort_cat(bio_dev_score,cat,thre = 2.5))
print(sort_cat(bio_dev_score,cat,thre = 3.5))

,category,actual,deviation
0,food,0,9.090909
1,fashion,1,4.545455
2,makeup,1,4.545455
3,beauty,0,9.090909
4,lifestyle,0,9.090909
5,luxury,0,9.090909
6,traveler,0,9.090909
7,art,1,4.545455
8,photography,0,9.090909
9,fitness,0,9.090909


['gaming', 'entertainment', 'technology']
['gaming', 'entertainment', 'technology']
['gaming', 'entertainment', 'technology']


In [58]:
check_result = pd.DataFrame(columns = ['user_id','handle','caption category','bio category'])

record = {'user_id':biorandomframe.at[0,'user_id'],'handle':biorandomframe.at[0,'handle'],'caption category':str(sort_cat(sd,cat,thre = 2.5)),'bio category':str(sort_cat(bio_dev_score,cat,thre = 2.5))}
check_result = check_result.append(record,ignore_index=True)
display(check_result)

,user_id,handle,caption category,bio category
0,165653,mythri_iyer,"['food', 'beauty']","['gaming', 'entertainment', 'technology']"
